# Prédiction de l’achalandage des stations Bixi

Ce projet vise à prédire le taux d’entrées et de sorties par tranche horaire dans les stations Bixi à l’aide de plusieurs modèles :

- **STGNN (Spatio-Temporal Graph Neural Network)** : réseau neuronal à graphes spatio-temporels, stations comme nœuds, arêtes basées sur la distance ou le temps de déplacement.
- **LSTM (Long Short-Term Memory)** : réseau neuronal récurrent adapté aux séries temporelles.
- **Modèle statistique** : régression ou analyse classique pour servir de baseline.

Nous comparerons la performance de ces modèles sur les données d’achalandage.

---

## 1. Importation des librairies et des données

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# Pour apprentissage
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader, Dataset
# Pour LSTM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# Pour le modèle statistique
# import statsmodels.api as sm
# Pour le graphe
# import networkx as nx

## 2. Exploration et préparation des données

- Nettoyage et visualisation

- Construction du graphe des stations

In [4]:
df = pd.read_csv("Data/merged2018v2.csv", parse_dates=['start_date', 'end_date'])
df['start_date'] = pd.to_datetime(df['start_date'])
# print(df.get(0))
df_grouped = df.groupby([ pd.Grouper(key="start_date", freq="15min"),
                 pd.Grouper('start_station_code')
                 ]).count()

print(df_grouped.sort_values(by=['end_date'], ascending=False))

                                        end_date  end_station_code  \
start_date          start_station_code                               
2018-07-18 22:30:00 7077                     109               109   
2018-07-21 22:30:00 7077                      92                92   
2018-07-07 22:30:00 7077                      86                86   
2018-07-14 22:30:00 7077                      86                86   
2018-07-11 22:30:00 7077                      81                81   
...                                          ...               ...   
2018-11-15 23:30:00 7021                       1                 1   
                    6907                       1                 1   
                    6394                       1                 1   
                    6386                       1                 1   
2018-04-10 12:15:00 6261                       1                 1   

                                        duration_sec  is_member  
start_date          sta

## 3. Modélisation

### 3.1 Modèle statistique (baseline) 

In [29]:
df = pd.read_csv("data/merged2018_reduced_dataset.csv") 
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
df['slot_15'] = df['end_date'].dt.floor('15T') # Tronquer à l'intervalle de 15 minutes le plus proche

# Calcul des arrivées par station et par intervalle de 15 minutes
arrivals = (
    df
    .groupby(['end_station_code', 'slot_15'])
    .size()
    .reset_index(name='arrivals')
)

# Calcul de la moyenne historique des arrivées par station, jour de la semaine, heure et minute
arrivals['dow'] = arrivals['slot_15'].dt.dayofweek   # 0=lundi
arrivals['hour'] = arrivals['slot_15'].dt.hour     # 0-23
arrivals['minute'] = arrivals['slot_15'].dt.minute   # 0,15,30,45

# Calcul de la moyenne historique des arrivées
historical_mean = (
    arrivals
    .groupby(['end_station_code', 'dow', 'hour', 'minute'])['arrivals']
    .mean()
    .reset_index()
    .rename(columns={'arrivals': 'mean_arrivals'})
)
# Fusion des données réelles avec les moyennes historiques
pred = arrivals.merge(
    historical_mean,
    on=['end_station_code', 'dow', 'hour', 'minute'],
    how='left'
)

from sklearn.metrics import mean_absolute_error, mean_squared_error
# Calcul des métriques MAE et RMSE
mae = mean_absolute_error(pred['arrivals'], pred['mean_arrivals'])
rmse = np.sqrt(mean_squared_error(pred['arrivals'], pred['mean_arrivals']))
print("MAE:", mae)
print("RMSE:", rmse)

MAE: 0.4473305597783349
RMSE: 0.6870355561229784


### 3.2 Modèle LSTM

### 3.3 Modèle STGNN (Spatio-Temporal Graph Neural Network)

## 4. Comparaison des performances

- Métriques : RMSE, MAE, etc.
- Visualisation des résultats